In [2]:
import pandas as pd
import plotly.express as px
import numpy as np
import torch_geometric as pyg
import torch
import networkx as nx
import pytorch_lightning as pl

In [47]:
climbs = pd.read_csv("data/climbs.csv")
grades = pd.read_csv("data/difficulty_grades.csv")
stats = pd.read_csv("data/climb_stats.csv")
holds = pd.read_csv("data/holes.csv")
holds = holds[holds['product_id'] == 1]
placements = pd.read_csv("data/placements.csv")
hole_to_hold = placements.set_index("hole_id")['hold_id'].to_dict()
holds['hole_id'] = holds['id']
holds['hold_id'] = holds['id'].apply(lambda x:hole_to_hold[x])
holds.set_index("hold_id", inplace=True)

In [48]:
df = pd.merge(climbs.drop("angle", axis=1), stats, left_on="uuid", right_on="climb_uuid")
df['display_grade'] = df['display_difficulty'].apply(lambda x: grades.loc[int(x) + 1, "boulder_name"])
df['average_grade'] = df['difficulty_average'].apply(lambda x: grades.loc[int(x) + 1, "boulder_name"])
df = df[df['layout_id'] == 1]

In [43]:
swooped = df[df["name"] == "swooped"].iloc[0]["frames"]
x_min, x_max = holds.x.min(), holds.x.max()
y_min, y_max = holds.y.min(), holds.y.max()
x_range, y_range = x_max - x_min, y_max - y_min


def encode_frame(frames: str, angle: int, difficulty: float) -> torch.Tensor:
    matrix = torch.zeros((46, 43, 4), dtype=torch.long)
    for frame in frames.split("p")[1:]:
        hold_id, color = frame.split("r")
        hold_id, color = int(hold_id), int(color)
        hold = holds.loc[hold_id]
        x, y = (hold.x - x_min) // 4, (hold.y - y_min) // 4


encode_frame(swooped, 40, 10)

tensor([[[0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         ...,
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0]],

        [[0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         ...,
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0]],

        [[0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         ...,
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0]],

        ...,

        [[0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         ...,
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0]],

        [[0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         ...,
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0]],

        [[0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         ...,
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0]]])

In [27]:
encode_frame()

-56 -12


'p1080r15p1110r15p1131r12p1146r12p1164r13p1202r13p1246r13p1250r13p1282r13p1331r13p1351r13p1385r14'